In [1]:
from __future__ import division, print_function, unicode_literals
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import pandas as pd
import re

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

## Preprocess the data

In [17]:
# TODO (maybe) : split on songs

file_path = "data/taylor_swift_lyrics.csv"

# Read the dataset from csv
df = pd.read_csv(filepath_or_buffer=file_path,
                encoding = "ISO-8859-1",
                sep=";")

# Extract the lyrics column
lyrics = df['lyric']

# Remove all special characters and add the words to a list
all_chars = []
for line in lyrics:
    for char in list(line):
        c = re.sub('[^A-Za-z0-9\s]+', '', char)
        all_chars.append(c)
    all_chars.append("\n")

# Get all unique characters from the dataset
unique_chars = list(set(all_chars))
        
# Create translation tables from char -> ind and ind -> char for one-hot encoding
chars_to_ind = dict((c, i) for i, c in enumerate(unique_chars))
ind_to_chars = dict((i, c) for i, c in enumerate(unique_chars))

print("All chars: ", len(all_chars))
print("Unique chars: ", len(unique_chars))

#for i in range(1000):
#    print(all_chars[i], end="")

for char in chars_to_ind.keys():
    print(char)
    
print(chars_to_ind)

All chars:  173542
Unique chars:  63

h
X
A
r
x
3
g
s
p
S
 
H
f
m
q
t
0
v
E
c
I


d
O
F
j
U
4
8
9
7
u
e
z
l
V
b
2
T
w
i
W
y
o
Y
k
R
1
B
Q
n
K
D
a
G
J
M
N
P
5
L
C
{'': 0, 'h': 1, 'X': 2, 'A': 3, 'r': 4, 'x': 5, '3': 6, 'g': 7, 's': 8, 'p': 9, 'S': 10, ' ': 11, 'H': 12, 'f': 13, 'm': 14, 'q': 15, 't': 16, '0': 17, 'v': 18, 'E': 19, 'c': 20, 'I': 21, '\n': 22, 'd': 23, 'O': 24, 'F': 25, 'j': 26, 'U': 27, '4': 28, '8': 29, '9': 30, '7': 31, 'u': 32, 'e': 33, 'z': 34, 'l': 35, 'V': 36, 'b': 37, '2': 38, 'T': 39, 'w': 40, 'i': 41, 'W': 42, 'y': 43, 'o': 44, 'Y': 45, 'k': 46, 'R': 47, '1': 48, 'B': 49, 'Q': 50, 'n': 51, 'K': 52, 'D': 53, 'a': 54, 'G': 55, 'J': 56, 'M': 57, 'N': 58, 'P': 59, '5': 60, 'L': 61, 'C': 62}


Now, we use the `dyanmic_rnn` to build the above RNN model.

Let's train an RNN to classify MNIST images. We will treat each image as a sequence of 28 rows of 28 pixels each (since each MNIST image is 28×28 pixels). Assume we will use cells of 100 recurrent neurons, plus a fully connected layer containing 10 neurons connected to the output of the last time step, followed by a softmax layer.

In [ ]:
# multi-layer RNN network for mnist
reset_graph()

n_steps = 28
n_inputs = 28
n_neurons = 100
n_layers = 3
n_outputs = 10
learning_rate = 0.001
n_epochs = 20
batch_size = 50

########################################
# loading dataset
########################################
mnist = input_data.read_data_sets("/tmp/data/")
X_test = mnist.test.images.reshape((-1, n_steps, n_inputs))
y_test = mnist.test.labels.astype("int")

########################################
# defining variables and placeholders
########################################
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])

########################################
# building the model
########################################
with tf.name_scope("rnn"):
    layers = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu) for layer in range(n_layers)]
    multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers)
    outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)    
    states_concat = tf.concat(axis=1, values=states)
    logits = tf.layers.dense(states_concat, n_outputs)

########################################
# defining the loss function
########################################
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy)
    
########################################
# training the model
########################################
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

########################################
# defining the evaluation metrics
########################################
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

########################################
# executing the model
########################################
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            X_batch = X_batch.reshape((-1, n_steps, n_inputs))
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

We build the above network again for the MNIST dataset, but this time we use LTSM instead of regular RNN.

In [ ]:
# building an LSTM RNN
reset_graph()

n_steps = 28
n_inputs = 28
n_neurons = 150
n_outputs = 10
n_layers = 3
learning_rate = 0.001
n_epochs = 10
batch_size = 150

########################################
# loading dataset
########################################
mnist = input_data.read_data_sets("/tmp/data/")
X_test = mnist.test.images.reshape((-1, n_steps, n_inputs))
y_test = mnist.test.labels.astype("int")

########################################
# defining variables and placeholders
########################################
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])

########################################
# building the model
########################################
with tf.name_scope("rnn"):
    lstm_cells = [tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons) for layer in range(n_layers)]
    multi_cell = tf.contrib.rnn.MultiRNNCell(lstm_cells)
    outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)
    top_layer_h_state = states[-1][1]
    logits = tf.layers.dense(top_layer_h_state, n_outputs, name="softmax")

########################################
# defining the loss function
########################################
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
########################################
# training the model
########################################
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)
    
########################################
# defining the evaluation metrics
########################################
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
########################################
# executing the model
########################################
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            X_batch = X_batch.reshape((batch_size, n_steps, n_inputs))
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)